In [1]:
import torch
import numpy as np
import requests
import sys

if "utils" in sys.modules:
    del sys.modules["utils"]
from utils import MahjongGame4P, mjg2imgstate, MahjongDataset

In [2]:
# Increasing layers

from torchsummary import summary


class ModelV1(torch.nn.Module):
    def __init__(self):
        super(ModelV1, self).__init__()
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()
        self.max_pool = torch.nn.MaxPool2d(3)
        
        self.conv2d_1 = torch.nn.Conv2d(98, 256, (6, 7), padding=(0, 3))
        self.bn_1 = torch.nn.BatchNorm2d(256)
        
        self.conv2d_2 = torch.nn.Conv2d(256, 256, (6, 7), padding=(0, 3))
        self.bn_2 = torch.nn.BatchNorm2d(256)
        
        self.conv2d_3 = torch.nn.Conv2d(256, 256, (6, 7), padding=(0, 3))
        self.bn_3 = torch.nn.BatchNorm2d(256)
        
        self.conv2d_4 = torch.nn.Conv2d(256, 512, 3, padding=1)
        self.bn_4 = torch.nn.BatchNorm2d(512)
        
        self.conv2d_5 = torch.nn.Conv2d(512, 1024, 3, padding=1)
        self.bn_5 = torch.nn.BatchNorm2d(1024)
        
        self.fc_1 = torch.nn.Linear(9216, 256)
        self.bn_6 = torch.nn.BatchNorm1d(256)
        
        self.fc_2 = torch.nn.Linear(256, 4)
    
    def forward(self, x):
        x = self.bn_1(self.relu(self.conv2d_1(x)))
        x = self.bn_2(self.relu(self.conv2d_2(x)))
        x = self.bn_3(self.relu(self.conv2d_3(x)))
        x = self.bn_4(self.relu(self.conv2d_4(x)))
        x = self.max_pool(x)
        x = self.bn_5(self.relu(self.conv2d_5(x)))
        x = self.flatten(x)
        x = self.bn_6(self.relu(self.fc_1(x)))
        x = self.fc_2(x)
        return x

summary(ModelV1(), (98, 24, 9), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 19, 9]       1,053,952
              ReLU-2           [-1, 256, 19, 9]               0
       BatchNorm2d-3           [-1, 256, 19, 9]             512
            Conv2d-4           [-1, 256, 14, 9]       2,752,768
              ReLU-5           [-1, 256, 14, 9]               0
       BatchNorm2d-6           [-1, 256, 14, 9]             512
            Conv2d-7            [-1, 256, 9, 9]       2,752,768
              ReLU-8            [-1, 256, 9, 9]               0
       BatchNorm2d-9            [-1, 256, 9, 9]             512
           Conv2d-10            [-1, 512, 9, 9]       1,180,160
             ReLU-11            [-1, 512, 9, 9]               0
      BatchNorm2d-12            [-1, 512, 9, 9]           1,024
        MaxPool2d-13            [-1, 512, 3, 3]               0
           Conv2d-14           [-1, 102

In [3]:
model = ModelV1()
model = torch.nn.DataParallel(model)
model.load_state_dict(torch.load("models/model.pth"))
model.to("cuda")
model.eval()

DataParallel(
  (module): ModelV1(
    (relu): ReLU()
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (max_pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (conv2d_1): Conv2d(98, 256, kernel_size=(6, 7), stride=(1, 1), padding=(0, 3))
    (bn_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2d_2): Conv2d(256, 256, kernel_size=(6, 7), stride=(1, 1), padding=(0, 3))
    (bn_2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2d_3): Conv2d(256, 256, kernel_size=(6, 7), stride=(1, 1), padding=(0, 3))
    (bn_3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2d_4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn_4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2d_5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn_5): Bat

In [4]:
LOG_HTML_FORMAT = "https://tenhou.net/0/log/?{log}"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"}

game_text = requests.get(LOG_HTML_FORMAT.format(log="2024021222gm-0001-0000-30c7666f"), headers=HEADERS).text

In [58]:
import copy
from pprint import pprint

game = MahjongGame4P(game_text)
game.select_round(0)
for _ in range(40):
    game.step(verbose=False)
while True:
    action, end = game.step(verbose=False)
    
    if action["action"] == "draw":
        game.render()
        tiles_throwable = sorted(game.hai[game.turn] + [game.tile_held[game.turn]])
        imgstate = {}
        for tile in tiles_throwable:
            move = {"action": "throw", "who": game.turn, "tile": tile}
            game_copy = copy.deepcopy(game)
            game_copy.step(move, verbose=False)
            
            for key, value in mjg2imgstate(game_copy).items():
                value = np.expand_dims(value, 0).astype(np.float32)
                if key in imgstate.keys():
                    imgstate[key] = np.concatenate([imgstate[key], value], axis=0)
                else:
                    imgstate[key] = value
                    
        data = MahjongDataset([], load=False)
        data.y = imgstate.pop("y")
        data.x_dict = imgstate
        data.post_load()
    
        x = torch.from_numpy(data[:][0]).to("cuda")
        pred = model(x)#.cpu().detach().numpy() * 25000
        pred = torch.softmax(pred[:, 0], dim=0)
        y = data[:][1] * 25000
        choice_dict = {game.id2str(tile): pred[n] for n, tile in enumerate(tiles_throwable)}
        pprint(choice_dict)
        break
    if end:
        break

<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀕🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東1局<br>0本場0供託</div>            	<div id=east_stats><b>東家 25000</b></div>            	<div id=east_river>🀙🀡🀄︎🀀🀌<br><br></div>            	<div id=east_hand>🀇🀉🀉🀎🀏🀝🀝🀔🀖🀘🀘🀃🀃 🀐</div>            	<div id=east_meld></div>            	<div id=south_stats>南家 25000</div>            	<div id=south_river>🀃🀂🀁🀀🀅<br><br></div>            	<div id=south_hand>🀇🀇🀋🀍🀎🀏🀟🀠🀠🀡🀒🀒🀔</div>            	<div id=south_meld></div>            	<div id=west_stats>西家 25000</div>            	<div id=west_river>🀁🀗🀓🀛🀂<br><br></div>            	<div id=west_hand>🀇🀈🀉🀉🀋🀍🀛🀝🀞🀕🀖🀗🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>北家 25000</div>            	<div id=north_river>🀂🀐🀀🀙🀠<br><br></div>            	<div id=north_hand>🀈🀊🀎🀎🀚🀜🀟🀠<red>🀔</red>🀖🀖🀗🀘</div>            	<div id=north_meld></div>

{'🀃': tensor(0.0734, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀇': tensor(0.0690, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀉': tensor(0.0732, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀎': tensor(0.0714, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀏': tensor(0.0705, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀐': tensor(0.0703, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀔': tensor(0.0728, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀖': tensor(0.0692, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀘': tensor(0.0730, device='cuda:0', grad_fn=<SelectBackward0>),
 '🀝': tensor(0.0688, device='cuda:0', grad_fn=<SelectBackward0>)}


In [11]:
game = MahjongGame4P(game_text)
game.select_round(1)
game.step(verbose=False)

throwable_tiles = (game.hai[game.turn] + [game.tile_held[game.turn]])
potential_scores = []

for tile in throwable_tiles:
    game = MahjongGame4P(game_text)
    game.select_round(1)
    game.step()
    
    move = {"action": "throw", "who": game.turn, "tile": tile}
    game.step(move)
    
    imgstate = mjg2imgstate(game)
    for key in imgstate.keys():
        imgstate[key] = np.expand_dims(imgstate[key], 0).astype(np.float32)
    
    data = MahjongDataset([], load=False)
    data.y = imgstate.pop("y")
    data.x_dict = imgstate
    data.post_load()
    
    x = torch.from_numpy(data[:][0]).to("cuda")
    pred = model(x).cpu().detach().numpy() * 25000
    y = data[:][1] * 25000
    print(f"Prediction: {pred}")
    potential_scores.append(pred[0, 0])
    # print(f"Actual val: {y}")
    
    game.render()

{'action': 'draw', 'who': 1, 'tile': 17}
{'action': 'throw', 'who': 1, 'tile': 8}
Prediction: [[ 3342.2485    613.4584   -191.92323 -3613.0168 ]]


<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀀🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東2局<br>1本場1供託</div>            	<div id=east_stats>北家 23500</div>            	<div id=east_river><br><br></div>            	<div id=east_hand>🀊🀊🀋🀝🀐🀑🀓🀗🀘🀘🀂🀆🀅</div>            	<div id=east_meld></div>            	<div id=south_stats><b>東家 25500</b></div>            	<div id=south_river>🀉<br><br></div>            	<div id=south_hand>🀋🀎🀚🀛🀛🀡🀐🀒🀗🀁🀃🀃🀆</div>            	<div id=south_meld></div>            	<div id=west_stats>南家 23500</div>            	<div id=west_river><br><br></div>            	<div id=west_hand>🀇🀋🀚🀝🀝🀞🀐🀑<red>🀔</red>🀖🀀🀁🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>西家 26500</div>            	<div id=north_river><br><br></div>            	<div id=north_hand>🀇🀌🀍🀜🀡🀐🀑🀓🀕🀕🀖🀁🀂</div>            	<div id=north_meld></div>

{'action': 'draw', 'who': 1, 'tile': 17}
{'action': 'throw', 'who': 1, 'tile': 29}
Prediction: [[ 3653.7285     -88.339836   827.5857   -4269.744   ]]


<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀀🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東2局<br>1本場1供託</div>            	<div id=east_stats>北家 23500</div>            	<div id=east_river><br><br></div>            	<div id=east_hand>🀊🀊🀋🀝🀐🀑🀓🀗🀘🀘🀂🀆🀅</div>            	<div id=east_meld></div>            	<div id=south_stats><b>東家 25500</b></div>            	<div id=south_river>🀎<br><br></div>            	<div id=south_hand>🀉🀋🀚🀛🀛🀡🀐🀒🀗🀁🀃🀃🀆</div>            	<div id=south_meld></div>            	<div id=west_stats>南家 23500</div>            	<div id=west_river><br><br></div>            	<div id=west_hand>🀇🀋🀚🀝🀝🀞🀐🀑<red>🀔</red>🀖🀀🀁🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>西家 26500</div>            	<div id=north_river><br><br></div>            	<div id=north_hand>🀇🀌🀍🀜🀡🀐🀑🀓🀕🀕🀖🀁🀂</div>            	<div id=north_meld></div>

{'action': 'draw', 'who': 1, 'tile': 17}
{'action': 'throw', 'who': 1, 'tile': 42}
Prediction: [[ 2524.8457   417.365    996.8427 -3770.3079]]


...

<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀀🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東2局<br>1本場1供託</div>            	<div id=east_stats>北家 23500</div>            	<div id=east_river><br><br></div>            	<div id=east_hand>🀊🀊🀋🀝🀐🀑🀓🀗🀘🀘🀂🀆🀅</div>            	<div id=east_meld></div>            	<div id=south_stats><b>東家 25500</b></div>            	<div id=south_river>🀃<br><br></div>            	<div id=south_hand>🀉🀋🀎🀚🀛🀛🀡🀐🀒🀗🀁🀃🀆</div>            	<div id=south_meld></div>            	<div id=west_stats>南家 23500</div>            	<div id=west_river><br><br></div>            	<div id=west_hand>🀇🀋🀚🀝🀝🀞🀐🀑<red>🀔</red>🀖🀀🀁🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>西家 26500</div>            	<div id=north_river><br><br></div>            	<div id=north_hand>🀇🀌🀍🀜🀡🀐🀑🀓🀕🀕🀖🀁🀂</div>            	<div id=north_meld></div>

{'action': 'draw', 'who': 1, 'tile': 17}
{'action': 'throw', 'who': 1, 'tile': 127}
Prediction: [[ 3983.7817   -338.71478   395.90536 -3890.961  ]]


<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀀🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東2局<br>1本場1供託</div>            	<div id=east_stats>北家 23500</div>            	<div id=east_river><br><br></div>            	<div id=east_hand>🀊🀊🀋🀝🀐🀑🀓🀗🀘🀘🀂🀆🀅</div>            	<div id=east_meld></div>            	<div id=south_stats><b>東家 25500</b></div>            	<div id=south_river>🀆<br><br></div>            	<div id=south_hand>🀉🀋🀎🀚🀛🀛🀡🀐🀒🀗🀁🀃🀃</div>            	<div id=south_meld></div>            	<div id=west_stats>南家 23500</div>            	<div id=west_river><br><br></div>            	<div id=west_hand>🀇🀋🀚🀝🀝🀞🀐🀑<red>🀔</red>🀖🀀🀁🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>西家 26500</div>            	<div id=north_river><br><br></div>            	<div id=north_hand>🀇🀌🀍🀜🀡🀐🀑🀓🀕🀕🀖🀁🀂</div>            	<div id=north_meld></div>

{'action': 'draw', 'who': 1, 'tile': 17}
{'action': 'throw', 'who': 1, 'tile': 17}
Prediction: [[ 3795.9001   -197.22229   567.81335 -4030.5015 ]]


<style>red{background:radial-gradient(red,black);-webkit-background-clip:text;-webkit-text-fill-color:transparent;}            rot{width:1.59em;transform:rotate(270deg)translateX(0.23em);display:inline-block;margin:0em -0.23em 0em -0.36em;}            #table{background-color:lightgray;color:black;line-height:1em;position:relative;font-size:30px;width:500px;height:500px;}            #dora_shown{font-size:25px;position:absolute;top:40%;left:50%;transform:translateX(-50%);}            #game_stats{line-height:1.2em;font-size:15px;text-align:center;position:absolute;top:48%;left:50%;transform:translateX(-50%);}            #east_stats{font-size:15px;position:absolute;top:60%;left:50%;transform:translateX(-50%);}            #east_river{position:absolute;top:65%;left:50%;transform:translateX(-2.45em);}            #east_hand{position:absolute;bottom:10%;left:0%;transform:translateX(250px)translateX(-50%);}            #east_meld{position:absolute;bottom:0.5%;right:0.5%;}            #south_stats{font-size:15px;position:absolute;bottom:50%;left:60%;transform:translateY(50%)translateX(-50%)rotate(270deg)translateY(50%);}            #south_river{position:absolute;bottom:50%;right:35%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #south_hand{position:absolute;bottom:50%;right:10%;transform:translateY(50%)translateX(50%)rotate(270deg)translateY(-50%);}            #south_meld{position:absolute;top:0.5%;right:0.5%;transform:translateY(-50%)translateX(50%)rotate(270deg)translateX(-50%)translateY(-50%);}            #west_stats{font-size:15px;position:absolute;bottom:60%;left:50%;transform:translateX(-50%)rotate(180deg);}            #west_river{position:absolute;bottom:65%;right:50%;transform:rotate(180deg)translateX(-2.45em);}            #west_hand{position:absolute;top:10%;left:0%;transform:translateX(250px)translateX(-50%)rotate(180deg);}            #west_meld{position:absolute;top:0.5%;left:0.5%;transform:rotate(180deg);}            #north_stats{font-size:15px;position:absolute;top:50%;right:60%;transform:translateY(-50%)translateX(50%)rotate(90deg)translateY(50%);}            #north_river{position:absolute;top:50%;left:35%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(50%)translateX(50%)translateX(-2.45em);}            #north_hand{position:absolute;top:50%;left:10.0%;transform:translateY(-50%)translateX(-50%)rotate(90deg)translateY(-50%);}            #north_meld{position:absolute;bottom:0.5%;left:0.5%;transform:translateY(50%)translateX(-50%)rotate(90deg)translateX(-50%)translateY(-50%);}</style><div id=table><div id=dora_shown>🀀🀫🀫🀫🀫</div>        	<div id=game_stats>四麻赤<br>東2局<br>1本場1供託</div>            	<div id=east_stats>北家 23500</div>            	<div id=east_river><br><br></div>            	<div id=east_hand>🀊🀊🀋🀝🀐🀑🀓🀗🀘🀘🀂🀆🀅</div>            	<div id=east_meld></div>            	<div id=south_stats><b>東家 25500</b></div>            	<div id=south_river>🀋<br><br></div>            	<div id=south_hand>🀉🀎🀚🀛🀛🀡🀐🀒🀗🀁🀃🀃🀆</div>            	<div id=south_meld></div>            	<div id=west_stats>南家 23500</div>            	<div id=west_river><br><br></div>            	<div id=west_hand>🀇🀋🀚🀝🀝🀞🀐🀑<red>🀔</red>🀖🀀🀁🀂</div>            	<div id=west_meld></div>            	<div id=north_stats>西家 26500</div>            	<div id=north_river><br><br></div>            	<div id=north_hand>🀇🀌🀍🀜🀡🀐🀑🀓🀕🀕🀖🀁🀂</div>            	<div id=north_meld></div>

In [12]:
max_score = max(potential_scores)
potential_scores.index(max_score)

9

In [13]:
throwable_tiles

[8, 29, 42, 45, 46, 69, 75, 83, 100, 114, 120, 123, 127, 17]